In [3]:
import torch
from dgl.sampling import sample_neighbors
from utils.utils import get_process_graph
from pyvis.network import Network

In [4]:
graph,features,_ = get_process_graph('ogbn-arxiv', -1)

randomize = True

batch = torch.randperm(graph.num_nodes()) if randomize else torch.arange(graph.num_nodes())

169343 1166243


In [5]:
def sample(graph, batch_in, no_layers):
    layers = []
    blocks = []
    # Note. Dont forget self loops otherwise GAT doesnt work.
    # Create bipartite graphs for the first l-1 layers
    last_layer = batch_in
    last_layer = last_layer.unique()
    layers.append(last_layer)
    iterations = 3
    fanout_factor = 1.5
    for i in range(no_layers):
        fanout = int(10/(i+1))
        print(fanout,i,len(last_layer))
        dest_ids,src_ids = sample_neighbors(graph, last_layer, fanout).edges()
        self_loop_dests = torch.cat([last_layer, dest_ids])
        edges = self_loop_dests, torch.cat([last_layer, src_ids])
        last_layer = torch.unique(self_loop_dests)
        layers.append(last_layer)
        blocks.append(edges)
    return layers,blocks

In [14]:
num_layers = 4
batch_size = 32
layers,blocks = sample(graph, batch[0:batch_size], num_layers)

for offset in range(0,num_layers):

  l1 = layers[offset].tolist()
  l2 = layers[offset+1].tolist()
  b2,b1 = map(lambda x: x.tolist(),blocks[offset])

  l1Max = max(l1)+1
  l2Added = [x + l1Max for x in l2]
  b2Added = [x + l1Max for x in b2]

  l1Label = [str(x) for x in l1]
  l2Label = [str(x) for x in l2]

  colors = ['red','blue','green','purple']

  l1Colors = [colors[features[x]] for x in l1]
  l2Colors = [colors[features[x]] for x in l2]

  net = Network(notebook=True, directed=True)
  
  net.add_nodes(l1,label=l1Label,color=l1Colors,shape=['triangle']*len(l1))
  net.add_nodes(l2Added,label=l2Label,color=l2Colors)
    
  net.add_edges(list(zip(b1,b2Added)))
  
  # net.toggle_physics(False)

  filename = "graphs/graph {}.html".format(offset)
  net.show(filename)
  print("Saved to {}".format(filename))

10 0 32
5 1 151
3 2 379
2 3 662
Saved to graphs/graph 0.html
Saved to graphs/graph 1.html
Saved to graphs/graph 2.html
Saved to graphs/graph 3.html
